In [20]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [17]:
if "rag" in pc.list_indexes():
    pc.delete_index("rag")
else:
    print("Index 'rag' does not exist.")


Index 'rag' does not exist.


In [18]:
pc = Pinecone (api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"))

In [19]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Anderson',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Dr. Anderson is knowledgeable but her lectures can be a bit dry at times. Overall, a solid professor.'},
 {'professor': 'Dr. Mark Wilson',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Absolutely fantastic professor! Always willing to help and makes difficult concepts easy to understand.'},
 {'professor': 'Dr. Sarah Johnson',
  'subject': 'Mathematics',
  'stars': 3,
  'review': 'Good professor but sometimes hard to follow. Make sure you attend the office hours.'},
 {'professor': 'Dr. James Parker',
  'subject': 'History',
  'stars': 5,
  'review': 'His lectures are engaging and he always ties history back to current events, which makes the material more interesting.'},
 {'professor': 'Dr. Michael Turner',
  'subject': 'Chemistry',
  'stars': 2,
  'review': 'Very hard to follow. The exams are tough and not reflective of the lecture material.'},
 {'professor': 'Dr. Olivia Harris',
  'su

In [28]:
processed_data = []
client = OpenAI()


for review in data ['reviews']:
    responce = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = responce.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
        }
    })

In [29]:
processed_data[0]

{'values': [-0.028197492,
  -0.014852299,
  -0.025256202,
  0.03437177,
  0.004399781,
  0.04467844,
  -0.005004447,
  0.016359407,
  0.021828748,
  0.013393809,
  -0.018936075,
  -0.007535537,
  -0.024174487,
  6.404636e-05,
  0.025207587,
  0.030847084,
  0.006514593,
  -0.016772646,
  -0.009097338,
  0.021366892,
  0.01600694,
  -0.01538708,
  0.01577601,
  -0.024867272,
  -0.015143998,
  -0.041226678,
  0.036608122,
  0.01892392,
  0.010719908,
  0.014730759,
  0.08094625,
  -0.017696356,
  0.026763309,
  -0.02902397,
  -0.049078222,
  0.058679953,
  0.04161561,
  0.039938346,
  0.008349861,
  0.017902976,
  0.01505892,
  0.006654365,
  -0.012543023,
  0.011278997,
  0.05551989,
  -0.023858482,
  -0.011509924,
  -0.014232441,
  0.03680259,
  0.03169787,
  -0.077688955,
  -0.004734019,
  0.043949194,
  -0.020905036,
  -0.0472308,
  0.028003028,
  0.021306122,
  0.0067333668,
  0.01939793,
  -0.0074808435,
  0.06553486,
  -0.028537806,
  0.03211111,
  -0.03850416,
  -0.043511648,
  -

In [31]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [33]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}